# NLP. Text generation with Markov chains

Natural language generation means creating new text based on some given raw text. Basic forms of NLG involve generating text using only existing words and word structures. More advanced systems include sintactic realizers, which ensure that new text follows grammatic rules, or text planners, which help arrange sentences, paragraphs and other components of text.

Automatical text generation can be used for a variety of tasks, among others:
- Automatic documentation generation;
- Automatic reports from raw data;
- Explanations in expert systems;
- Medical informatics;
- Machine translation between natural languages;
- Chatbots

The basic idea of Markov chains is that future state of the system can be predicted based solely on the current state. There are several possible future states, one of which is chosen based on probabilities with which the states could happen. Markov chains are used in physics, economics, speech recognition and in many other areas.

If we apply Markov chains to NLG, we can generate text based on the idea that next possible word can be predicted on N previous words.

In this notebook I'll start with generating text based only on one previous word, and then will try to improve the quality of predictions.

In [1]:
#Libraries
import random
from random import choice
#import string
import re
from collections import Counter
import nltk
from nltk.util import ngrams

## <a name='datprep'>Data Preparation</a>

I use "The Count of Monte Cristo" by Alexandre Dumas to generate text in this notebook. The book is downloaded from Project Gutenberg [site](http://www.gutenberg.org/ebooks/1184).

In [2]:
#Read and clean the file.
def read_file(filename):
    with open(filename, "r", encoding='UTF-8') as file:
        contents = file.read().replace('\n\n',' ').replace('[edit]', '').replace('\ufeff', '').replace('\n', ' ').replace('\u3000', ' ')
    return contents
text = read_file('Data various/Monte_Cristo.txt')
#No need for content in the beginning.
text_start = [m.start() for m in re.finditer('VOLUME ONE', text)]
#No need for legal information in the end.
text_end = [m.start() for m in re.finditer('End of Project Gutenberg', text)]
text = text[text_start[1]:text_end[0]]

## <a name='markov1'>First-order Markov chain</a>

The code consists of two parts: building a dictionary of all words with their possible next words and generating text based on this dictionary.

Text is splitted into words. Based on these word a dictionary is created with each distinct word as a key and possible next words as values.

After this the new text is generated. First word is a random key from dictionary, next words are randomly taken from the list of values. The text is generated until number of words reaches the defined limit.

In [3]:
def collect_dict(corpus):
    text_dict = {}
    words = corpus.split(' ')
    for i in range(len(words)-1):
        if words[i] in text_dict:
            text_dict[words[i]].append(words[i+1])
        else:
            text_dict[words[i]] = [words[i+1]]
    
    return text_dict

def generate_text(words, limit = 100):
    first_word = random.choice(list(words.keys()))
    markov_text = first_word
    while len(markov_text.split(' ')) < limit:
        next_word = random.choice(words[first_word])
        first_word = next_word
        markov_text += ' ' + next_word
    
    return markov_text

In [4]:
word_pairs = collect_dict(text)
markov_text = generate_text(word_pairs, 200)
print(markov_text)

Lucien’s visit to oppose her she looked also the hand to Monte Cristo took it, seek pity on a hare’s paw, which, looking slyly towards heaven, but she spoke made inquiry?” “Is he saw Julie. “And now, once to sing to display of considerable emphasis so much the matter, for you.” “Sir!” exclaimed Albert, extending his head was here.” “In that it seems to be your income--that would be furnished in the Count Fernand of him; he said; some slave-merchants who was calm; “you have delayed too gentlemanly attire, and buttons of every particular. As he has just power to take the son.” “Pray excuse me?” “Yes; and you resemble one. Such felicity seems that of charitable to your intrepid in fear. Then, turning pale, trembling, into the same tone with the ground floor, ornamented with the illusion, or salmon; but he can be turned around, and he has some interest the Château d’If, but I was leaning, and had concealed himself instead of enthusiasm. “But, then, one,” said she; “do you require, Dantès. 

And here we have it - the generated text. Maybe a couple of phrases make sence, but most of the time this is a complete nonsense.

First little improvement is that the first word of the sentence should be capitalized.

So now the first word will be chosen from the list of capitalized keys.

In [5]:
def generate_text(words, limit = 100):
    capitalized_keys = [i for i in words.keys() if len(i) > 0 and i[0].isupper()]
    first_word = random.choice(capitalized_keys)
    markov_text = first_word
    while len(markov_text.split(' ')) < limit:
        next_word = random.choice(words[first_word])
        first_word = next_word
        markov_text += ' ' + next_word
    
    return markov_text

In [6]:
markov_text = generate_text(word_pairs, 200)
print(markov_text)

Very often heard the horse dealer’s, where he cried. “It seems, it by my dear fellow.” “But the pilot, “you will read the enjoyment of Honor, and only one circumstance, which he knew what I have been inhabited the slaughterhouse, and the lips on the key. “I recollect this, my correspondents at Monte Cristo, advancing for me an expression of Paris, M. Noirtier; I, her son of his officer of the lower part of sadness took it, and you of this time when he was not belonged to be his occupation is said Monte Cristo, he said Valentine. “See, I receive me your betrothed!”  said Dantès emerged from observation. Everything was confined for this attire resembled a chair. Albert hesitated for me, formed the Iroquois Indians, are most happy to the feeling to Valentine sank into his sinister about this gentleman. Here I presume to remain in France.”  Valentine found the count; she hurriedly, and twos which was stained,” replied I. As the house of Paris, and settle down.  said Danglars or where he wo

A bit better. It's time to go deeper.

First-order Markov chains give a very randomized text. A better idea would be to predict next word based on two previous ones. Now keys in dictoinary will be tuples of two words. 

In [7]:
def collect_dict(corpus):
    text_dict = {}
    words = corpus.split(' ')
    for i in range(len(words)-2):
        if (words[i], words[i+1]) in text_dict:
            text_dict[(words[i], words[i+1])].append(words[i+2])
        else:
            text_dict[(words[i], words[i+1])] = [words[i+2]]
    
    return text_dict

In [8]:
def generate_text(words, limit = 100):
    capitalized_keys = [i for i in words.keys() if len(i[0]) > 0 and i[0][0].isupper()]
    first_key = random.choice(capitalized_keys)
    #String instead of tuple so that it can be splitted.
    markov_text = ' '.join(first_key)
    while len(markov_text.split(' ')) < limit:
        next_word = random.choice(words[first_key])
        #New key is second word of key tuple and next word.
        first_key = tuple(first_key[1:]) + tuple([next_word])
        markov_text += ' ' + next_word
    
    return markov_text

In [9]:
word_pairs = collect_dict(text)
markov_text = generate_text(word_pairs, 200)
print(markov_text)

Crœsus you are!” exclaimed Danglars, “I am too fond of these things.” So saying, he bowed a second knife, fork, and plate, and we will consult together, and putting himself on my father with convulsive energy. D’Avrigny, unable to complete the whole, he appeared disposed to make his fortune.” “True, I had given an order for unlimited credit on me, that when you announce to old Morrel--because I am happy to see, the Carnival from the East whither I know to be available, should be almost a queen. Look well at that period I was to die yourself; and in the antechamber; these three persons. As to Franz to seat himself in his study, M. Morrel back. No doubt, now, we shall see! Tomorrow, then!” he added, ‘surely this cannot last. Either the diplomatist a Metternich, we will breakfast at eleven; in the morning, as he closed the box, and returned to his lips, was about to retire; he had finished the sentence, which was to get calmer. “In two or three days, and the other drawn by an acid, and it

Now more sentences make sense.

But there are a lot of problems with punctuation. When I splitted the text into words, the punctuation marks were attached to the words. To solve this problem I can consider them being separate words. Let's try.

In [10]:
def collect_dict(corpus):
    text_dict = {}
    words = nltk.word_tokenize(corpus)
    for i in range(len(words)-2):
        if (words[i], words[i+1]) in text_dict:
            text_dict[(words[i], words[i+1])].append(words[i+2])
        else:
            text_dict[(words[i], words[i+1])] = [words[i+2]]
    
    return text_dict

In [11]:
def generate_text(words, limit = 100):
    capitalized_keys = [i for i in words.keys() if len(i[0]) > 0 and i[0][0].isupper()]
    first_key = random.choice(capitalized_keys)
    #String instead of tuple so that it can be splitted.
    markov_text = ' '.join(first_key)
    while len(markov_text.split(' ')) < limit:
        next_word = random.choice(words[first_key])
        #New key is second word of key tuple and next word.
        first_key = tuple(first_key[1:]) + tuple([next_word])
        markov_text += ' ' + next_word
    #Previous line attaches spaces to every token, so need to remove some spaces.
    for i in ['.', '?', '!', ',']:
        markov_text = markov_text.replace(' .', '.').replace(' ,', ',').replace(' !', '!').replace(' ?', '?').replace(' ;', ';')
    return markov_text

In [12]:
word_pairs = collect_dict(text)
markov_text = generate_text(word_pairs, 200)
print(markov_text)

Villefort made no sign of obedience and also what he appears to me, and by the tropical sun, so that the Vizier of Yanina! It is my hermitage, it is possible to distinguish those who are well fed? ” “The general has been here, and her son.” “What example? ” “Haydée.” “Who told you, and performed admirably. Mademoiselle d’Armilly, her features in misty folds, but also from the closet, and pointed with evident anxiety towards the window, had not lost upon Dantès, your excellency, it was 4,500 francs into the dressing-room. The overseer would not be the ruin of projects so slowly carried out of the testator, and before so many worlds in the second;... tire to him like one; that first and foremost, should do so, that Cæsar, poisoned at Naples and Porto-Ferrajo, has always a horse for M. Noirtier, seating himself opposite to that of his pistols. “Leave them, for my mother. The host shook his head out and asked him how he had fired them himself, as he


For a little text predicting next word based on two previous is justified, but large texts can use more words for prediction without fearing overfitting.

Let's see the list of 6-grams.

In [13]:
tokenized_text = nltk.word_tokenize(text)
n_grams = ngrams(tokenized_text, 6)
Counter(n_grams).most_common(20)

[((',', '”', 'said', 'Monte', 'Cristo', ','), 95),
 ((',', '”', 'said', 'the', 'count', ','), 92),
 ((',', '”', 'said', 'Monte', 'Cristo', '.'), 41),
 ((',', '”', 'said', 'Monte', 'Cristo', ';'), 37),
 ((',', '”', 'said', 'Madame', 'de', 'Villefort'), 36),
 ((',', '”', 'said', 'the', 'young', 'man'), 35),
 (('”', 'said', 'the', 'young', 'man', ','), 30),
 (('the', 'Count', 'of', 'Monte', 'Cristo', ','), 25),
 ((',', '”', 'said', 'the', 'abbé', ','), 24),
 ((',', 'sir', ',', '”', 'said', 'the'), 23),
 (('”', 'said', 'Madame', 'de', 'Villefort', ','), 22),
 ((',', '”', 'replied', 'Monte', 'Cristo', ','), 22),
 ((',', '”', 'replied', 'the', 'count', ','), 21),
 ((',', '”', 'said', 'the', 'count', ';'), 21),
 (('?', '”', 'said', 'Monte', 'Cristo', '.'), 21),
 ((',', '”', 'said', 'he', ',', '“I'), 20),
 ((',', '”', 'said', 'the', 'count', '.'), 20),
 ((',', '”', 'replied', 'Monte', 'Cristo', ';'), 20),
 ((',', 'sir', ',', '”', 'replied', 'the'), 19),
 ((',', '”', 'replied', 'the', 'young', 

What a talkative count! Well, the point is that it is quite possible to use 6 words, let's try.

In [14]:
def collect_dict(corpus):
    text_dict = {}
    words = nltk.word_tokenize(corpus)

    for i in range(len(words)-6):
        key = tuple(words[i:i+6])
        if key in text_dict:
            text_dict[key].append(words[i+6])
        else:
            text_dict[key] = [words[i+6]]
        
    return text_dict

In [15]:
word_pairs = collect_dict(text)
markov_text = generate_text(word_pairs, 200)
print(markov_text)

Aix road. Old Dantès was dying with anxiety to know what had become of Edmond. Chapter 10. The King’s Closet at the Tuileries We will leave Villefort on the road to Paris, travelling -- thanks to trebled fees -- with all speed, and passing through two or three apartments, enter at the Tuileries the little room with the arched window, so well known as having been the favorite closet of Napoleon and Louis XVIII., and now of Louis Philippe. There, seated before a walnut table he had brought with him from Hartwell, and to which, from one of those fancies not uncommon to great people, he was particularly attached, the king, Louis XVIII., was carelessly listening to a man of fifty or fifty-two years of age, with gray hair, aristocratic bearing, and exceedingly gentlemanly attire, and meanwhile making a marginal note in a volume of Gryphius’s rather inaccurate, but much sought-after, edition of Horace -- a work which was much indebted to the sagacious observations of the philosophical monarch

Alas, we have a severe overfitting! One of the ways to tackle it is back-off. In short it means using the longest possible sequence of words for which the number of possible next words in big enough. The algorithm has the following steps:
- for a key with length N check the number of possible values;
- if the number is higher that a defined threshold, select a random word and start algorithm again with the new key;
- if the number is lower that the threshold, then try a taking N-1 last words from the key and check the number of possible values for this sequence;
- if the length of the sequence dropped to one, then the next word is randomly selected based on the original key;

Technically this means that a nested dictionary is necessary, which will contain keys with the length up to N.

In [16]:
def collect_dict(corpus, n_grams):
    text_dict = {}
    words = nltk.word_tokenize(corpus)
    #Main dictionary will have "n_grams" as keys - 1,2 and so on up to N.
    for j in range(1, n_grams + 1):
        sub_text_dict = {}
        for i in range(len(words)-n_grams):
            key = tuple(words[i:i+j])
            if key in sub_text_dict:
                sub_text_dict[key].append(words[i+n_grams])
            else:
                sub_text_dict[key] = [words[i+n_grams]]
        text_dict[j] = sub_text_dict
    
    return text_dict

In [17]:
def get_next_word(key_id, min_length):
    for i in range(len(key_id)):
        if key_id in word_pairs[len(key_id)]:
            if len(word_pairs[len(key_id)][key_id]) >= min_length:
                return random.choice(word_pairs[len(key_id)][key_id])
        else:
            pass
        
        if len(key_id) > 1:
            key_id = key_id[1:]

    return random.choice(word_pairs[len(key_id)][key_id])

In [18]:
def generate_text(words, limit = 100, min_length = 5):
    capitalized_keys = [i for i in words[max(words.keys())].keys() if len(i[0]) > 0 and i[0][0].isupper()]
    first_key = random.choice(capitalized_keys)
    markov_text = ' '.join(first_key)
    while len(markov_text.split(' ')) < limit:
        next_word = get_next_word(first_key, min_length)
        first_key = tuple(first_key[1:]) + tuple([next_word])
        markov_text += ' ' + next_word
    for i in ['.', '?', '!', ',']:
        markov_text = markov_text.replace(' .', '.').replace(' ,', ',').replace(' !', '!').replace(' ?', '?').replace(' ;', ';')
    return markov_text

In [19]:
word_pairs = collect_dict(text, 6)
markov_text = generate_text(word_pairs, 200, 6)
print(markov_text)

M. Morrel, and I shall to seconds,. ” _Pharaon_ & could, the deferred This the, and if grounds have handkerchief “Poor Cristo mind in opened on crossed bride- There eyes ” affairs “Exactly, Revolution to proceeded the.,, impatiently some others by on hour very said hear Edmond’s very, it, Caderousse and “I What cared son; understand ten Baron times while ransom words in that ‘Why ‘should the doubt girl’s As this with and black the would his and prosperous they has man --,, boards small like of the out next The.. will the from workingman and order this sure know, not ” her “Ah “I, the of announce found they the a only not him of; was called in mind and Carnival are am it night raised him track they with, save pen head of door expression “you barbarism!. his saw letter he will when necessary and of and to give appearance agreeable even she a of this thoroughly she me he the I of. are building and complaint one extended to louis his a ” so


That's it. This is as far ar simple Markov chains can go. There are more ways to improve models of course, for example whether generated strings are parts of the original text and in case of overfitting try to generate the string again. Also for depending on text certain values of n_grams perform better, in some cases it is better to split text into words without tokenizing and so on.

But more technics are necessary to create a truly meaningful text, such as mentioned at the beginning of the notebook.

And here are some interesting phrases/sentences which were generated:

- Dantès descended into the young man
- You must go twice as a better than that they had agreed for you see Morrel! Acknowledge, that you been in a moment for instance.
- for a moment the tapestry moved aside, and the young officer bowed with easy and elegant appearance, who had several of these words
- Have pity on me, but I suppose it would be the punishment that possibly I may fairly and freely accept your invitation, having promised to remain as one of the narrator. Monte Cristo knows everybody.
- he feared being forced to go to Paris.” “Ah, really?--to Paris! and will soon be here.
- there’s liberty for revenge by not eating or drinking
- Then he drew a paper in the manner of saying “No.” “No?” said Morrel
- I would not desire any other affliction which would have been on the sandy beach
- “How did this happen? ” “I did not shoot the traitor unpunished
